#**Ingesta a través de API rest**

#**Configuración de AWS CLI en Colab**


In [1]:
##Instalación de boto3 & request para la API de los datos
!pip install boto3
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.1 MB/s eta 0:00:00


In [2]:
import requests
import boto3
import json
from io import StringIO
import pandas as pd

In [3]:
##Parametros de conexión con AWS###
!pip install awscli
!pip install --upgrade awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1


In [4]:
##Parametros del bucket##
s3_bucket='rawzonetrabajo1'
s3_target_file_name='raw/atm_con.csv'

In [5]:
session = boto3.Session(
     aws_access_key_id=aws_access_key_id,
     aws_secret_access_key=aws_secret_access_key,
     aws_session_token=aws_session_token
 )
s3 = session.client('s3')

NameError: name 'aws_access_key_id' is not defined

#**Obteniendo la información del API que tiene el IMF**

####**Annual Surface Temperature Change**

In [6]:
##Consumiendo la API
##Esta se encuentra en un formato GeoJSON
url_atm_con = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_2_Climate_Indicators_Monthly_Atmospheric_Carbon_Dioxide_concentrations/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

#Realizamos el request
response_atm_con = requests.get(url_atm_con)
data_atm_con= response_atm_con.json()

##Los archivos GeoJSON son tipo de estructura de datos que son frecuentemente utilizados para datos geográficos
##Estos consisten en varios elementos principales, con roles especificos.
##Para este caso estamos usando el elemento de "features" y "properties". El primero es un array utilizado para almacenar propiedades de los objetos
##El segundo contiene data no geométrica sobre las propiedades, aquí se encuentra la metadata
features_atm_con = data_atm_con['features']

##Normalizamos el JSON
df_atm_con= pd.json_normalize(features_atm_con, record_path=None, meta=['id', 'geometry'], errors='ignore')

##Lo almacenamos en un DataFrame de Pandas
df_atm_con = pd.DataFrame([feature.get('properties', {}) for feature in features_atm_con])

In [7]:
##Vamos a convertir el DataFrame a CSV en memoria para el posterior almacenamiento en S3
csv_buffer = StringIO()
df_atm_con.to_csv(csv_buffer)

In [9]:
df_atm_con.to_csv('atm_con.csv', index=False)

In [ ]:
##Enviamos los datos a S3
s3.put_object(Body=csv_buffer.getvalue(),Bucket=s3_bucket,Key=s3_target_file_name)